<a href="https://colab.research.google.com/github/e-chong/Singapore-Ship-Detection/blob/master/Download_Sentinel_2_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Sentinel-2 Imagery

This notebook downloads sentinel-2 imagery

# 1. Setup Environment

In [0]:
# install libraries

# suppress output
%%capture 

!pip install rasterio
!pip install geopandas
!pip install sentinelsat
!pip install satpy
!pip install glymur
!pip install python-geotiepoints
!pip install pyspectral

In [0]:
# Load Dependencies
import numpy as np
import pandas as pd
import geopandas as gpd # vector spatial operations
import rasterio as rio # raster spatial operations
from rasterio.plot import show #convenience wrapper for pyplot.imshow for RGB and 1 band images

#packages for Sentinel-2 I/O
from sentinelsat import SentinelAPI
from satpy.scene import Scene
from satpy import find_files_and_readers

import requests, zipfile, io, os, shutil # downloading and extracting zipfolders

import matplotlib.pyplot as plt # plotting

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
DRIVE_PATH = '/content/drive/My Drive/MUSA-650-Final-Project/'
#Load param file
with open(DRIVE_PATH+'params.json', 'r') as file:
    params = json.load(file)
    
    SENTINEL_USER = params['SENTINEL_USER']
    SENTINEL_PWD = params['SENTINEL_PWD']
    KAGGLE_USER = params['KAGGLE_USER']
    KAGGLE_KEY = params['KAGGLE_KEY']

In [0]:
# Sentinel API Authentication
tile_id = "48NUG" # unique Sentinel-2 tile id for images covering Singapore
api = SentinelAPI(SENTINEL_USER, SENTINEL_PWD, 'https://scihub.copernicus.eu/dhus')

queryResults = api.to_geodataframe(api.query(tileid=tile_id,
                                             date=('20100101','NOW'),
                                             platformname='Sentinel-2',
                                             cloudcoverpercentage=(0, 6)))
scenes = queryResults

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
queryResults.head()

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,cloudcoverpercentage,sensoroperationalmode,tileid,hv_order_tileid,format,processingbaseline,platformname,filename,instrumentname,instrumentshortname,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,level1cpdiidentifier,uuid,granuleidentifier,datastripidentifier,geometry
8345c17f-dd76-4398-b4bf-2eb708631b07,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-12-27T03:21:31.024Z, Instrument: MS...",2019-12-27 03:21:31.024,2019-12-27 03:21:31.024,2019-12-27 03:21:31.024,2019-12-27 08:03:15.620,23567,118,3.6098,INS-NOBS,48NUG,NG48U,SAFE,02.08,Sentinel-2,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,Multi-Spectral Instrument,MSI,747.76 MB,GS2A_20191227T032131_023567_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,S2A_OPER_MSI_L1C_TL_EPAE_20191227T060736_A0235...,8345c17f-dd76-4398-b4bf-2eb708631b07,NaN,NaN,"MULTIPOLYGON (((103.20277 0.81602, 104.18934 0..."
5d57f964-03cb-4001-a2f2-33f9eba9605e,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-07-05T03:15:49.024Z, Instrument: MS...",2019-07-05 03:15:49.024,2019-07-05 03:15:49.024,2019-07-05 03:15:49.024,2019-07-05 09:41:20.667,12156,118,5.5518,INS-NOBS,48NUG,NG48U,SAFE,02.07,Sentinel-2,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,Multi-Spectral Instrument,MSI,743.30 MB,GS2B_20190705T031549_012156_N02.07,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,S2B_OPER_MSI_L1C_TL_SGS__20190705T065323_A0121...,5d57f964-03cb-4001-a2f2-33f9eba9605e,NaN,NaN,"MULTIPOLYGON (((103.20277 0.81602, 104.18934 0..."
06a584ce-37d8-4b9c-a35c-9ce4924d81c2,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-04-06T03:15:49.024Z, Instrument: MS...",2019-04-06 03:15:49.024,2019-04-06 03:15:49.024,2019-04-06 03:15:49.024,2019-04-06 09:28:17.321,10869,118,5.8114,INS-NOBS,48NUG,NG48U,SAFE,02.07,Sentinel-2,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,Multi-Spectral Instrument,MSI,767.45 MB,GS2B_20190406T031549_010869_N02.07,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,S2B_OPER_MSI_L1C_TL_EPAE_20190406T073031_A0108...,06a584ce-37d8-4b9c-a35c-9ce4924d81c2,S2B_OPER_MSI_L1C_TL_EPAE_20190406T073031_A0108...,S2B_OPER_MSI_L1C_DS_EPAE_20190406T073031_S2019...,"MULTIPOLYGON (((103.20277 0.81602, 104.18934 0..."
69ff6d59-6ecd-4de2-9dad-85e23a8d0dbe,S2B_MSIL1C_20190327T031729_N0207_R118_T48NUG_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-27T03:17:29.024Z, Instrument: MS...",2019-03-27 03:17:29.024,2019-03-27 03:17:29.024,2019-03-27 03:17:29.024,2019-03-27 14:06:18.415,10726,118,5.9415,INS-NOBS,48NUG,NG48U,SAFE,02.07,Sentinel-2,S2B_MSIL1C_20190327T031729_N0207_R118_T48NUG_2...,Multi-Spectral Instrument,MSI,781.26 MB,GS2B_20190327T031729_010726_N02.07,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20190327T031729_N0207_R118_T48NUG_2...,S2B_OPER_MSI_L1C_TL_SGS__20190327T083636_A0107...,69ff6d59-6ecd-4de2-9dad-85e23a8d0dbe,S2B_OPER_MSI_L1C_TL_SGS__20190327T083636_A0107...,S2B_OPER_MSI_L1C_DS_SGS__20190327T083636_S2019...,"MULTIPOLYGON (((103.31888 0.81606, 104.18934 0..."
0fda02a0-abba-4b57-8be0-9d00e8241930,S2B_MSIL1C_20190327T031729_N0207_

In [0]:
tempList = list()
for uuid in scenes.uuid:
  tempList.append(api.get_product_odata(uuid, full=True))

sceneMeta = pd.DataFrame(tempList)
sceneMeta = sceneMeta.loc[sceneMeta['Online'] == True] # filter out images that are in Long Term Archive and have a 30 minute throttle to download

sceneMeta

,id,title,size,md5,date,footprint,url,Online,Creation Date,Ingestion Date,Cloud cover percentage,Datatake sensing start,Date,Degraded ancillary data percentage,Degraded MSI data percentage,Filename,Footprint,Format,Format correctness,General quality,Generation time,Geometric quality,Identifier,Instrument,Instrument abbreviation,Instrument mode,Instrument name,JTS footprint,Mission datatake id,NSSDC identifier,Orbit number (start),Pass direction,Platform serial identifier,Processing baseline,Processing level,Product type,Radiometric quality,Relative orbit (start),Satellite,Satellite name,Satellite number,Sensing start,Sensing stop,Sensor quality,Size,Tile Identifier,Tile Identifier horizontal order,Level-1C PDI Identifier,Datastrip identifier,Granule identifier
0,8345c17f-dd76-4398-b4bf-2eb708631b07,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,784125118,FCD2A37B4A069C5342299AD2AEE90B41,2019-12-27 03:21:31.024,"POLYGON((103.20205689083804 1.808922187860351,...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,True,2019-12-27 08:04:08.105,2019-12-27 08:03:15.620,3.6098,2019-12-27 03:21:31.024,2019-12-27 03:21:31.024,0.0,0,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,PASSED,PASSED,2019-12-27 06:07:36,PASSED,S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_2...,MSI,MSI,INS-NOBS,Multi-Spectral Instrument,MULTIPOLYGON (((103.2027660714231 0.8160247595...,GS2A_20191227T032131_023567_N02.08,2015-028A,23567,DESCENDING,Sentinel-2A,2.08,Level-1C,S2MSI1C,PASSED,118,Sentinel-2,Sentinel-2,A,2019-12-27 03:21:31.024,2019-12-27 03:21:31.024,PASSED,747.76 MB,48NUG,NG48U,S2A_OPER_MSI_L1C_TL_EPAE_20191227T060736_A0235...,NaN,NaN
1,5d57f964-03cb-4001-a2f2-33f9eba9605e,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,779449455,5E51BCC8A45CF3ADD026433463D37D58,2019-07-05 03:15:49.024,"POLYGON((103.20205689083804 1.808922187860351,...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,True,2019-07-05 10:32:33.443,2019-07-05 09:41:20.667,5.5518,2019-07-05 03:15:49.024,2019-07-05 03:15:49.024,0.0,0,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,PASSED,PASSED,2019-07-05 06:53:23,PASSED,S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_2...,MSI,MSI,INS-NOBS,Multi-Spectral Instrument,MULTIPOLYGON (((103.2027660714231 0.8160247595...,GS2B_20190705T031549_012156_N02.07,2017-013A,12156,DESCENDING,Sentinel-2B,2.07,Level-1C,S2MSI1C,PASSED,118,Sentinel-2,Sentinel-2,B,2019-07-05 03:15:49.024,2019-07-05 03:15:49.024,PASSED,743.30 MB,48NUG,NG48U,S2B_OPER_MSI_L1C_TL_SGS__20190705T065323_A0121...,NaN,NaN
2,06a584ce-37d8-4b9c-a35c-9ce4924d81c2,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,804777353,98F454F41D8C8248E26F5DAC2A6C7526,2019-04-06 03:15:49.024,"POLYGON((103.20205689083804 1.808922187860351,...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,True,2019-04-06 09:29:13.485,2019-04-06 09:28:17.321,5.8114,2019-04-06 03:15:49.024,2019-04-06 03:15:49.024,0.0,0,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,PASSED,PASSED,2019-04-06 07:30:31,PASSED,S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_2...,MSI,MSI,INS-NOBS,Multi-Spectral Instrument,MULTIPOLYGON (((103.2027660714231 0.8160247595...,GS2B_20190406T031549_010869_N02.07,2017-013A,10869,DESCENDING,Sentinel-2B,2.07,Level-1C,S2MSI1C,PASSED,118,Sentinel-2,Sentinel-2,B,2019-04-06 03:15:49.024,2019-04-06 03:15:49.024,PASSED,767.45 MB,48NUG,NG48U,S2B_OPER_MSI_L1C_TL_EPAE_20190406T073031_A0108...,S2B_OPER_MSI_L1C_DS_EPAE_20190406T073031_S2019...,S2B_OPER_MSI_L1C_TL_EPAE_20190406T073031_A0108...
10,82d4edad-fe56-44f3-9e85-46449cb8d3c0,S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_2...,764374083,ff28b79dd0a50802664f65c48100e50a,2017-11-22 03:20:29.027,"POLYGON((103.20205689083804 1.808922187860351,...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,True,2017-11-22 20:41:54.999,2017-11-22 20:38:59.643,4.7439,2017-11-22 03:20:29.027,2017

In [0]:
# download sentinel images available online
for index, row in sceneMeta.iterrows():
  uuid = row['id']
  title = row['title']
  filename = row['Filename']
  print(f'Downloading {title}')
  api.download(uuid)

Downloading: 100%|██████████| 784M/784M [01:07<00:00, 11.6MB/s]
MD5 checksumming: 100%|██████████| 784M/784M [00:01<00:00, 459MB/s]


Downloading: 100%|██████████| 779M/779M [01:06<00:00, 11.8MB/s]
MD5 checksumming: 100%|██████████| 779M/779M [00:01<00:00, 465MB/s]


Downloading: 100%|██████████| 805M/805M [01:14<00:00, 10.9MB/s]
MD5 checksumming: 100%|██████████| 805M/805M [00:01<00:00, 465MB/s]


Downloading: 100%|██████████| 764M/764M [01:07<00:00, 11.3MB/s]
MD5 checksumming: 100%|██████████| 764M/764M [00:01<00:00, 461MB/s]


In [0]:
# # move zip folders to google drive if needed
# from google.colab import drive
# drive.mount('/content/drive')
# for index, row in sceneMeta.iterrows(): 
#     title = row['title']
#     filename = row['Filename']
#     print(f'Moving {title} to Google Drive')
#     shutil. copy ( title+'.zip' , '/content/drive/My Drive/'+title+'.zip' )
#     with zipfile.ZipFile(title+'.zip', 'r') as safezip:
#       safezip.extractall(filename)

In [0]:
# Unzip folders
# this creates SAFE files/directories which are explained here: https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/data-formats
for index, row in sceneMeta.iterrows(): 
    title = row['title']
    filename = row['Filename']
    print(f'Unzipping {title}')
    with zipfile.ZipFile(title+'.zip', 'r') as safezip:
      safezip.extractall(filename)

Unzipping S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736
Unzipping S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323
Unzipping S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031
Unzipping S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526


In [0]:
# NOTE: You can load all channels at once with:
# scn.load(scn.available_dataset_names()) # load all channels

# save RGB channels as a geotiff
for index, row in sceneMeta.iterrows():
  title = row['title']
  filename = row['Filename']
  print(f'Reading {title}')
  files = find_files_and_readers(base_dir=filename, reader="msi_safe")
  scn = Scene(filenames=files)
  scn.load(['true_color'])
  print(f'Saving {title} TIFF')
  scn.save_dataset('true_color', filename=title+'.tif')

Reading S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-chan

Saving S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736 TIFF


/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


Reading S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323
Saving S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323 TIFF
Reading S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031
Saving S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031 TIFF
Reading S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526
Saving S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526 TIFF


In [0]:
# delete uneeded files to free up space
# eventually maybe move into extraction for-loops
for name in sceneMeta.title:
  if os.path.exists(name+'.zip'):
    print(f'Deleting {name}.zip')
    os.remove(name+'.zip')
  else:
    pass
  if os.path.exists(name+'.SAFE'):
    print(f'Deleting {name}.SAFE')
    shutil.rmtree(name+'.SAFE')

Deleting S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736.zip
Deleting S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736.SAFE
Deleting S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323.zip
Deleting S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323.SAFE
Deleting S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031.zip
Deleting S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031.SAFE
Deleting S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526.zip
Deleting S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526.SAFE


In [0]:
os.listdir()

['.config',
 'S2B_MSIL1C_20190406T031549_N0207_R118_T48NUG_20190406T073031.tif',
 'S2A_MSIL1C_20191227T032131_N0208_R118_T48NUG_20191227T060736.tif',
 'S2B_MSIL1C_20171122T032029_N0206_R118_T48NUG_20171122T083526.tif',
 'S2B_MSIL1C_20190705T031549_N0207_R118_T48NUG_20190705T065323.tif',
 'sample_data']

Downloaded Sentinel-2 imagery manually from Google drive to mark ship locations as points in QGIS 